# Adapts the spillover matrix for use in CellProfiller

In [ ]:
library(CATALYST)
library(tiff)

##

In [ ]:
fn_sm = snakemake@input$fn_sm_mat
# a csv with a list of metals used, generated from the IMC preprocessing ipython notebook:
fn_imc_metals = list.files(snakemake@input$fol_csvs, pattern = '.*\\.csv$',full.names = T)[1]
fol_out = snakemake@output$fol_out
prefix_out =  '_sm'
fns_base = list.files(snakemake@input$fol_sm_files, pattern = snakemake@params$re_fns_full)

In [ ]:
dir.create(fol_out,recursive = T)

Write the spillover matrix for cellprofier

In [ ]:
sm = read.csv(fn_sm, row.names = 1)
analysis_channels = read.csv(fn_imc_metals,header = F)
analysis_channels = paste(as.character(analysis_channels$V1), 'Di', sep = '')

Adapt matrix to panel

In [ ]:
sm[sm>1]=1
sm = sm[row.names(sm) %in% analysis_channels,colnames(sm) %in% analysis_channels]
sm_table = CATALYST::adaptSpillmat(input_sm = as.matrix(sm), out_chs = analysis_channels)

Writes out a 32bit tiff that can be used in CellProfiller together with the "CorrectSpilloverApply" module

In [ ]:
for (fn in fns_base){
    fn_out = file.path(fol_out, sub('\\.tiff$', paste0(prefix_out,'.tiff'),fn))
    tiff::writeTIFF(sm_table, fn_out, bits.per.sample = 32, reduce = T)
}



In [ ]:
print(length(analysis_channels))
nrow(sm_table)
sm_table

In [ ]:
sessionInfo()